# JETANK ColorRecognition 颜色识别

此文档用来获得屏幕中某一块像素的HSV数值，可用于后续的颜色识别追踪功能，在此过程中你可以了解到HSV色彩空间与RGB色彩空间的区别和特性。

HSV(Hue, Saturation, Value)是根据颜色的直观特性由A. R. Smith在1978年创建的一种颜色空间, 也称六角锥体模型(Hexcone Model)。
这个模型中颜色的参数分别是：色调（H），饱和度（S），明度（V）。

与通常我们适用的RGB色彩空间相比，HSV更加人性化，符合人类视觉习惯，将某一个色彩使用三个参数来定义：
1.这是什么颜色：色调（H）
2.这个颜色的纯度高不高：饱和度（S）
3.这个颜色是亮还是暗：明度（V）

而RGB色彩模型只是更加机器友好，可以利用光色三原色原理，仅使用三种颜色的灯光亮暗程度来叠加成我们所需要的颜色。
所以在openCV的颜色识别功能中，为了避免环境白平衡以及光线亮暗的影响，我们使用HSV色彩模型来匹配颜色。


# 导入摄像头相关的函数库

运行以下代码块后，稍等片刻，等待摄像头初始化，初始化成功后会在该代码块的下方出现一个300x300大小的摄像头实时视频画面。

你可以在这个画面上点击右键，点击`Create New View for Output`，这样就可以将摄像头画面放置在窗口的一遍，即使浏览到该文档的后面部分，你依然可以随时观看摄像头的画面，该方法也同样适用于其它组件。

多次运行该代码块有一定概率会初始化失败，在`jetbot.Camera`中已经包含了该问题的解决方法，你只需要重新运行该Kernel即可，但是注意不要使用该选项卡上方的圆形箭头来重新运行，那种方法有一定概率会依然初始化失败。

建议重新运行该Kernel的方法：
在左侧`File Browser`中，前方有绿色圆点的`*.ipynb`文件上点击右键（前面有绿色的原点说明Kernel正在运行中），选择`Shut Down Kernel`，你会发现绿色的圆点消失了，然后关闭掉该选项卡再双击刚才被关掉的`*.ipynb`文件来重新运行kernel。

再次运行以下代码块，摄像头就应该可以正常初始化了。

In [1]:
import traitlets
import ipywidgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=300, height=300)

image_widget = ipywidgets.Image()  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

display(image_widget)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

# 将画面转换为HSV色彩空间

以下代码用来将画面转换为HSV色彩空间，并且在画面上添加一些元素方便对准需要采样的像素块。

注意，运行以下代码块后你所看到的画面并不会发生改变，当方法函数定义完成，最后改变`image_widget.value`的值并调用`camera.observe()`函数才会在实时画面中观察到最终的效果，前者用于显示处理过的画面，后者用于当采集到新的画面后立即调用图像处理的相关方法，所以在最后一个代码块运行前，你在实时视频的画面上看不到变化是正常的。

In [2]:
import cv2

# 在这个函数中导入采集到的画面帧，加以处理。
def colorSelect(imageInput):
    # 将导入的帧转换为HSV色彩空间。
    hsv = cv2.cvtColor(imageInput, cv2.COLOR_BGR2HSV)
    
    # 增加模糊效果，目的是为了减小画面噪点带来的误差。
    hsv = cv2.blur(hsv, (15, 15))
    
    # 获得中间像素点的HSV参数，由于画面的大小是300x300，所以中间像素点的位置为 (150, 150)。
    hsvValue = hsv[150, 150]
    
    # 在中间画一个20x20的举行，用来瞄准需要采样的物体。
    cv2.rectangle(imageInput,(140,140),(160,160),(255,255,255),1)
    
    # 在画面中显示中间点的HSV参数。
    cv2.putText(imageInput,"HSV:{}".format(hsvValue),(80,180), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255,255,255),1,cv2.LINE_AA)
    
    # 返回处理后的画面
    return imageInput

# 处理视频帧并显示

运行以下代码块后，你就可以看到画面的颜色发生了变化，说明视频画面已经通过`colorSelect()`函数处理成功。

In [3]:
def execute(change):
    global image_widget
    image = change['new']
    image_widget.value = bgr8_to_jpeg(colorSelect(image))
    
execute({'new': camera.value})
camera.unobserve_all()
camera.observe(execute, names='value')

视频画面中会有一个白色小方块，用这个小方块来对准你希望获取HSV数值的物体。
对准物体后，方块下方的HSV数值会在一定范围内变化，记录下这三个数值的最大值和最小值，例如：
颜色上限 = [44, 255, 255]
颜色下限 = [24, 100, 100]

## 问题：为什么需要这个数值快速变化而不是取样多次取平均或取多个点的平均值？
因为当经过模糊处理后，这个像素点实际上已经是和周围像素点平均后的结果了，我们需要考虑这一区域内单独像素点的数值跳跃范围，所以不能使用采集多个像素点再取平均的方案，那样会大幅度缩小实际画面中该颜色的HSV数值取值范围，影响后续的色彩识别相关功能的工作效率。

# 关闭图像处理和摄像头
运行以下代码块来关闭该图像处理功能。

In [4]:
camera.unobserve(execute, names='value')

在结束该例程前，我们需要关闭摄像头，这样才可以释放摄像头资源供其它例程使用。

In [4]:
camera.stop()